### This file is dedicated to extract median rental price per suburb in past data file

Created by Yuecheng Wang 24-9-2024

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
past_data = pd.ExcelFile("../../data/raw/domain/past_data.xlsx")

print("Available sheets:", past_data.sheet_names)


Available sheets: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house', 'All properties']


In [3]:
output_relative_dir = '../../data/raw/domain/past_data_sheets'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

In [4]:
# Function to clean the data by replacing '-' with the closest available value for each suburb
def fill_missing_with_closest(df):
    for i in range(df.shape[1]):
        column = df.iloc[:, i]
        column.replace('-', pd.NA, inplace=True)
        column.fillna(method='ffill', inplace=True)  # Forward fill
        column.fillna(method='bfill', inplace=True)  # Backward fill
    return df

# Function to convert time periods to decimals like 2000.00, 2000.25, 2000.50, and 2000.75
def convert_time_to_decimal(time_series):
    time_decimal = []
    for time in time_series:
        if pd.isna(time):
            continue
        time = str(time)
        year, month = int(time[-4:]), time[:3]
        if month == 'Mar':
            time_decimal.append(year)
        elif month == 'Jun':
            time_decimal.append(year + 0.25)
        elif month == 'Sep':
            time_decimal.append(year + 0.50)
        elif month == 'Dec':
            time_decimal.append(year + 0.75)
    return time_decimal

# Loop through each sheet in the Excel file
for sheet in past_data.sheet_names:
    print(f"Processing sheet: {sheet}")
    
    # Load the data from the current sheet
    all_properties_df = pd.read_excel(past_data, sheet_name=sheet)
    
    # Extract suburb names and relevant rent data
    suburb_col = all_properties_df.iloc[2:, 1]  # Suburb names
    median_rent_data = all_properties_df.iloc[2:, 84:].iloc[:, 1::2]  # Every second column starting from the 2nd one (median)
    
    # Fill missing values
    median_rent_data_cleaned = fill_missing_with_closest(median_rent_data)

    # Extract the years and convert them to decimal format
    time_periods = pd.Series(all_properties_df.iloc[0, 82:].iloc[::2])
    time_decimal = convert_time_to_decimal(time_periods)

    # Prepare data for each suburb
    suburb_rent_data = pd.DataFrame()
    for i, suburb in enumerate(suburb_col):
        rents = median_rent_data_cleaned.iloc[i].dropna().astype(float).values
        years = time_decimal[:len(rents)]
        if len(rents) == len(years):  # Only include matching year-rent pairs
            temp_df = pd.DataFrame({'suburb': suburb, 'year': years, 'median_rent': rents})
            suburb_rent_data = pd.concat([suburb_rent_data, temp_df], ignore_index=True)

    # Aggregating the data by taking the mean for duplicate suburb-year combinations
    suburb_rent_data = suburb_rent_data.groupby(['suburb', 'year'], as_index=False).median()

    # Pivot the data so that each row corresponds to a suburb and each column represents a year
    suburb_rent_data = suburb_rent_data.pivot(index='suburb', columns='year', values='median_rent')

    # Save the wide format data to a new CSV file, naming the file after the sheet
    data_file_path = f"{output_relative_dir}/{sheet} suburb rent data.csv"
    suburb_rent_data.to_csv(data_file_path, index=True)

    print(f"Data saved to {data_file_path}")


Processing sheet: 1 bedroom flat


/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='bfill', inplace=True)  # Backward fill


Data saved to ../../data/raw/domain/past_data_sheets/1 bedroom flat suburb rent data.csv
Processing sheet: 2 bedroom flat


/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='bfill', inplace=True)  # Backward fill


Data saved to ../../data/raw/domain/past_data_sheets/2 bedroom flat suburb rent data.csv
Processing sheet: 3 bedroom flat


...

/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='bfill', inplace=True)  # Backward fill


Data saved to ../../data/raw/domain/past_data_sheets/3 bedroom house suburb rent data.csv
Processing sheet: 4 bedroom house


/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='bfill', inplace=True)  # Backward fill


Data saved to ../../data/raw/domain/past_data_sheets/4 bedroom house suburb rent data.csv
Processing sheet: All properties
Data saved to ../../data/raw/domain/past_data_sheets/All properties suburb rent data.csv


/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_35110/3092081257.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  column.fillna(method='bfill', inplace=True)  # Backward fill
